In [1]:
import sys
print(sys.executable)
print(sys.version)


/app/cloned_repo/LLM-World/.venv/bin/python
3.11.14 (main, Dec 17 2025, 21:07:37) [Clang 21.1.4 ]


In [2]:
from transformers import BitsAndBytesConfig

from datasets import load_dataset

/app/cloned_repo/LLM-World/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [3]:
system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."

In [4]:
dataset_id = "euclid-multimodal/Geoperception"
dataset = load_dataset(dataset_id)

In [5]:
dataset_linecomparison = dataset['train'].filter(lambda x: x['predicate'] == 'LineComparison')
train_test = dataset_linecomparison.train_test_split(test_size=0.5,seed=42)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11657/11657 [00:34<00:00, 334.90 examples/s]


In [6]:
train_test

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'predicate', 'image'],
        num_rows: 697
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'predicate', 'image'],
        num_rows: 697
    })
})

In [7]:
# train_test['train']
# train_test['test']

In [8]:
def format_data(sample):
    return [
        {"role":"system", "content":[{"type":"text","text":system_message}]},
        {"role":"user", "content":[{"type":"image","image":sample['image']}]},
        {"role":"assistant","content":[{"type":"text","text":sample['answer']}]}
    ]

In [9]:
train_dataset = [format_data(x) for x in train_test['train']]
print(len(train_dataset))
train_dataset[:5]

697


[[{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=305x214>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'DB'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=555x450>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'UT'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. T

In [10]:
test_dataset = [format_data(x) for x in train_test['test']]
print(len(test_dataset))
test_dataset[:5]

697


[[{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=226x172>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'DB'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=453x338>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'YX'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. T

# Model

In [11]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor